<a href="https://colab.research.google.com/drive/1FeU5dUwyPAto4Sc9jBli5Pu1duIQX7hp">Abre este Jupyter en Google Colab</a>

# Visualización del conjunto de datos

En este notebook se muestran algunos de los mecanismos más utilizados para la visualización del conjunto de datos.

## Conjunto de datos

### Descripción
NSL-KDD is a data set suggested to solve some of the inherent problems of the KDD'99 data set which are mentioned in. Although, this new version of the KDD data set still suffers from some of the problems discussed by McHugh and may not be a perfect representative of existing real networks, because of the lack of public data sets for network-based IDSs, we believe it still can be applied as an effective benchmark data set to help researchers compare different intrusion detection methods. Furthermore, the number of records in the NSL-KDD train and test sets are reasonable. This advantage makes it affordable to run the experiments on the complete set without the need to randomly select a small portion. Consequently, evaluation results of different research work will be consistent and comparable.

### Ficheros de datos
* <span style="color:green">**KDDTrain+.ARFF**: The full NSL-KDD train set with binary labels in ARFF format</span>
* <span style="color:green">**KDDTrain+.TXT**: The full NSL-KDD train set including attack-type labels and difficulty level in CSV format</span>
* KDDTrain+_20Percent.ARFF:	A 20% subset of the KDDTrain+.arff file
* KDDTrain+_20Percent.TXT:	A 20% subset of the KDDTrain+.txt file
* KDDTest+.ARFF:	The full NSL-KDD test set with binary labels in ARFF format
* KDDTest+.TXT:	The full NSL-KDD test set including attack-type labels and difficulty level in CSV format
* KDDTest-21.ARFF:	A subset of the KDDTest+.arff file which does not include records with difficulty level of 21 out of 21
* KDDTest-21.TXT:	A subset of the KDDTest+.txt file which does not include records with difficulty level of 21 out of 21

### Descarga de los ficheros de datos
https://iscxdownloads.cs.unb.ca/iscxdownloads/NSL-KDD/#NSL-KDD

### Referencias adicionales sobre el conjunto de datos
_M. Tavallaee, E. Bagheri, W. Lu, and A. Ghorbani, “A Detailed Analysis of the KDD CUP 99 Data Set,” Submitted to Second IEEE Symposium on Computational Intelligence for Security and Defense Applications (CISDA), 2009._

## 1. Lectura del conjunto de datos

In [ ]:
# Lectura del conjunto de datos mediante funciones de Python
with open("datasets/NSL-KDD/KDDTrain+.txt") as train_set:
    df = train_set.readlines()
df

In [ ]:
# Lectura del conjunto de datos utilizando Pandas
import pandas as pd

df = pd.read_csv("datasets/NSL-KDD/KDDTrain+.txt")
df

In [ ]:
# Mostramos los ficheros en el directorio del conjunto de datos
import os

os.listdir("datasets/NSL-KDD/")

An **ARFF (Attribute-Relation File Format)** file is an ASCII text file that describes a list of instances sharing a set of attributes. ARFF files were developed by the Machine Learning Project at the Department of Computer Science of The University of Waikato for use with the Weka machine learning software. Más información: https://www.cs.waikato.ac.nz/ml/weka/arff.html

In [ ]:
# Instalamos un nuevo paquete externo para poder leer ficheros arff
!pip install liac-arff

In [ ]:
# Lectura del conjunto de datos que se encuentra en formato .arff
import arff

with open('datasets/NSL-KDD/KDDTrain+.arff', 'r') as train_set:
    df = arff.load(train_set)

df.keys()

In [ ]:
df["data"]

In [ ]:
df["attributes"]

In [ ]:
# Parseamos los atributos para obtener únicamente los nombres
atributos = [attr[0] for attr in df["attributes"]]
atributos

In [ ]:
# Leemos el conjunto de datos con Pandas para facilitar la manipulación
df = pd.DataFrame(df["data"], columns=atributos)
df

Llegados a este punto lo ideal es construir una función que permita leer el conjunto de datos de manera más limpia. Este tipo de prácticas son de gran utilidad para que nuestro código en el Jupyter Notebook sea más modular y pueda reutilizarse de manera más sencilla para futuros ejercicios.

In [ ]:
def load_kdd_dataset(data_path):
    """Lectura del conjunto de datos NSL-KDD."""
    with open(data_path, 'r') as train_set:
        dataset = arff.load(train_set)
    attributes = [attr[0] for attr in dataset["attributes"]]
    return pd.DataFrame(dataset["data"], columns=attributes)

In [ ]:
load_kdd_dataset('datasets/NSL-KDD/KDDTrain+.arff')

## 2. Funciones básicas de visualización de los datos

* El proceso de visualización siempre debe realizarse sobre el trainning set y apartando el test set. Esto evita que nuestro cerebro genere intuiciones del test set que podemos incorporar en nuestro modelo
* Una buena práctica es crear una copia del trainning set y jugar con ella. De esta manera, si realizamos transformaciones que dañan el tranning set, el original no se ve afectado

In [ ]:
# Lectura y copia del conjunto de datos
df_orig = load_kdd_dataset('datasets/NSL-KDD/KDDTrain+.arff')
df = df_orig.copy()

In [ ]:
# Mostrar en pantalla un número determinado de filas
df.head(10)

In [ ]:
# Mostrar información básica sobre el conjunto de datos
df.info()

In [ ]:
# Mostrar información estadística sobre el conjunto de datos
df.describe()

In [ ]:
# Mostrar los valores únicos que tiene un atributo determinado
df["protocol_type"].value_counts()

In [ ]:
df["class"].value_counts()

In [ ]:
# Mostrar los valores de la característica como un histograma
%matplotlib inline
import matplotlib.pyplot as plt
df["protocol_type"].hist()

In [ ]:
# Representar gráficamente la distribución de los atributos
df.hist(bins=50, figsize=(20,15))
plt.show()

## 3. Funciones avanzadas de visualización de los datos

### Buscando correlaciones

* Se puede calcular el coeficiente de correlación estándar para ver la correlación entre cada par de atributos
* El coeficiente de correlación, solo mide **correlaciones lineales**, esto quiere decir que si x va hacia arriba, mediría si y va hacia arriba o hacia abajo.
* **Hay que intentar buscar correlaciones sobre todo con el atributo objetivo (el que queremeos predecir), en este caso _class_**

In [ ]:
# El atributo class de nuestro conjunto de datos tiene valores categoricos
df["class"]

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Transformamos los valores del atributo class de categoricos a numéricos
labelencoder = LabelEncoder()
df["class"] = labelencoder.fit_transform(df["class"])

# Transformamos los valores de los atributos categóricos a numéricos
df["protocol_type"] = labelencoder.fit_transform(df["protocol_type"])
df["service"] = labelencoder.fit_transform(df["service"])
df["flag"] = labelencoder.fit_transform(df["flag"])

df

In [ ]:
# Mostrar la correlación entre los atributos del conjunto de datos
corr_matrix = df.corr()
corr_matrix["class"].sort_values(ascending=False)

In [ ]:
# Mostrar correlación lineal entre todos los atributos del conjunto de datos
df.corr()

In [ ]:
# Representar gráficamente la matriz de correlación
corr = df.corr()
fig, ax = plt.subplots(figsize=(8, 8))
ax.matshow(corr)
plt.xticks(range(len(corr.columns)), corr.columns);
plt.yticks(range(len(corr.columns)), corr.columns);

In [ ]:
# Representar gráficamente las correlaciones
from pandas.plotting import scatter_matrix

attributes = ["same_srv_rate", "dst_host_srv_count", "class", "dst_host_same_srv_rate"]

scatter_matrix(df[attributes], figsize=(12,8))
plt.show()